# Policy Router

> API Router for Generate Controls, Generate Policy, Control GAP Analyzer, Policy GAP Analyzer

In [ ]:
#| default_exp routers/policy

In [ ]:
#| export
from fastapi import APIRouter, FastAPI, HTTPException, Query, Header, Body, Response, Depends, File, Form, UploadFile, BackgroundTasks
from fastapi.responses import FileResponse, JSONResponse
from typing import List, Dict, Set, Tuple, Sequence, Optional, Any, TypeVar, Union, Callable, Type
import zipfile
import io
import uvicorn
import time
import asyncio
import json
import arrow
from app.modules import logger
from app.configs.settings import settings
from app.modules.chain import get_chain
from app.modules.common.utils import api_key_auth, validate_parse_input_file, generate_task, validate_parse_input_file_policy_gap
from app.configs.clients import SQLClient, BlobStorageClient, CustomException
from app.api_models.responses import root_response, health_response, success_response, failure_response, job_response, job_status_response
from app.policy.job_tracker import JobTracker
# from langchain.chat_models import ChatOpenAI
from langserve import add_routes
app = APIRouter()

In [ ]:
#| export
def run_async_generate_task(standards, job_id, job_type):
    "Converting sync from async, to run the job in background"
    asyncio.run(generate_task(standards, job_id,job_type))


In [ ]:
#| export
@app.post("/generate-policies", dependencies=[Depends(api_key_auth)])
async def generate_policies(background_tasks: BackgroundTasks, standard_zip: UploadFile = File(...)):
    "Policy Genius API Endpoint"
    try:
        logger.info(f"On generate-policies endpoint!")
        standards = await validate_parse_input_file(standard_zip)
        logger.info(f"Standards Generated!")
        job_request = {
            "operation": "create",
            "job_type": "generate-policies"
        }
        job_tracker = JobTracker()
        response = await job_tracker.generate_job_id(job_request)
        background_tasks.add_task(run_async_generate_task, standards, response['job_id'],job_request['job_type'])
        return JSONResponse(
            content = job_response(
            status = response['status'],
            job_id = response['job_id'],
            selflink = response['selflink']
        ).dict(),
        status_code = 200)
        # return {"filename": file_name, "content_type": standard_doc.content_type}
    except CustomException as e:
        error_string = f"Custom Exception: {e.message}"
        logger.error(error_string)
        return JSONResponse(
            content = failure_response(
                status = "Error",
                error_code = e.code,
                detail = error_string,
                trace_context = ""
            ).dict(),
            status_code = e.code
        )
    except Exception as e:
        error_string = f"Exception, Error: {str(e)}"
        logger.error(error_string)
        return JSONResponse(
            content = failure_response(
                status = "Error",
                error_code = 500,
                detail = error_string,
                trace_context = ""
            ).dict(),
            status_code = 500
        )


In [ ]:
#| export
@app.post("/generate-controls", dependencies=[Depends(api_key_auth)])
async def generate_controls(background_tasks: BackgroundTasks, standard_zip: UploadFile = File(...)):
    "Control Genius API Endpoint"
    try:
        logger.info(f"On generate-controls endpoint!")
        standards = await validate_parse_input_file(standard_zip)
        logger.info(f"Standards Generated!")
        job_request = {
            "operation": "create",
            "job_type": "generate-controls"
        }
        job_tracker = JobTracker()
        response = await job_tracker.generate_job_id(job_request)
        background_tasks.add_task(run_async_generate_task, standards, response['job_id'],job_request['job_type'])
        return JSONResponse(
            content = job_response(
            status = response['status'],
            job_id = response['job_id'],
            selflink = response['selflink']
        ).dict(),
        status_code = 200)
        # return {"filename": file_name, "content_type": standard_doc.content_type}
    except CustomException as e:
        error_string = f"Custom Exception: {e.message}"
        logger.error(error_string)
        return JSONResponse(
            content = failure_response(
                status = "Error",
                error_code = e.code,
                detail = error_string,
                trace_context = ""
            ).dict(),
            status_code = e.code
        )
    except Exception as e:
        error_string = f"Exception, Error: {str(e)}"
        logger.error(error_string)
        return JSONResponse(
            content = failure_response(
                status = "Error",
                error_code = 500,
                detail = error_string,
                trace_context = ""
            ).dict(),
            status_code = 500
        )

In [ ]:
#| export
@app.post("/policy-gap-analyzer", dependencies=[Depends(api_key_auth)])
async def control_gap_analyzer(background_tasks: BackgroundTasks, standard_zip: UploadFile = File(...)):
    "Policy Gap Analyzer API Endpoint"
    try:
        logger.info(f"On policy-gap-analyzer endpoint!")
        standards = await validate_parse_input_file_policy_gap(standard_zip)
        logger.info(f"Standards Generated!")
        job_request = {
            "operation": "create",
            "job_type": "policy-gap-analyzer"
        }
        job_tracker = JobTracker()
        response = await job_tracker.generate_job_id(job_request)
        background_tasks.add_task(run_async_generate_task, standards, response['job_id'],job_request['job_type'])
        return JSONResponse(
            content = job_response(
            status = response['status'],
            job_id = response['job_id'],
            selflink = response['selflink']
        ).dict(),
        status_code = 200)
        # return {"filename": file_name, "content_type": standard_doc.content_type}
    except CustomException as e:
        error_string = f"Custom Exception: {e.message}"
        logger.error(error_string)
        return JSONResponse(
            content = failure_response(
                status = "Error",
                error_code = e.code,
                detail = error_string,
                trace_context = ""
            ).dict(),
            status_code = e.code
        )
    except Exception as e:
        error_string = f"Exception, Error: {str(e)}"
        logger.error(error_string)
        return JSONResponse(
            content = failure_response(
                status = "Error",
                error_code = 500,
                detail = error_string,
                trace_context = ""
            ).dict(),
            status_code = 500
        )

In [ ]:
#| export
@app.post("/control-gap-analyzer", dependencies=[Depends(api_key_auth)])
async def control_gap_analyzer(background_tasks: BackgroundTasks, standard_zip: UploadFile = File(...)):
    "Control Gap Analyzer API Endpoint"
    try:
        logger.info(f"On control-gap-analyzer endpoint!")
        standards = await validate_parse_input_file(standard_zip, is_gap=True)
        logger.info(f"Standards Generated!")
        job_request = {
            "operation": "create",
            "job_type": "control-gap-analyzer"
        }
        job_tracker = JobTracker()
        response = await job_tracker.generate_job_id(job_request)
        background_tasks.add_task(run_async_generate_task, standards, response['job_id'],job_request['job_type'])
        return JSONResponse(
            content = job_response(
            status = response['status'],
            job_id = response['job_id'],
            selflink = response['selflink']
        ).dict(),
        status_code = 200)
        # return {"filename": file_name, "content_type": standard_doc.content_type}
    except CustomException as e:
        error_string = f"Custom Exception: {e.message}"
        logger.error(error_string)
        return JSONResponse(
            content = failure_response(
                status = "Error",
                error_code = e.code,
                detail = error_string,
                trace_context = ""
            ).dict(),
            status_code = e.code
        )
    except Exception as e:
        error_string = f"Exception, Error: {str(e)}"
        logger.error(error_string)
        return JSONResponse(
            content = failure_response(
                status = "Error",
                error_code = 500,
                detail = error_string,
                trace_context = ""
            ).dict(),
            status_code = 500
        )



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()